## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-25-06-55-03 +0000,bbc,Hot weather boosted retail sales in June,https://www.bbc.com/news/articles/c3353der4evo
1,2025-07-25-06-51-26 +0000,nyt,Australia Says Gaza Situation Beyond ‘Worst Fe...,https://www.nytimes.com/2025/07/25/world/middl...
2,2025-07-25-06-47-34 +0000,nyt,Thailand and Cambodia Clash in Deadly Border D...,https://www.nytimes.com/2025/07/24/world/asia/...
3,2025-07-25-06-46-03 +0000,nypost,Suspects in deaths of Kansas City Chiefs fans ...,https://nypost.com/2025/07/25/us-news/kansas-c...
4,2025-07-25-06-45-40 +0000,wapo,Arizona woman sentenced over $17 million North...,https://www.washingtonpost.com/nation/2025/07/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2334/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
92,trump,53
232,new,18
15,thailand,17
16,cambodia,16
7,gaza,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
66,2025-07-25-01-00-00 +0000,wsj,Streetwise: The Trump administration has rampe...,https://www.wsj.com/economy/central-banking/tr...,147
54,2025-07-25-02-10-00 +0000,wsj,Republican National Committee Chair Michael Wh...,https://www.wsj.com/politics/elections/north-c...,119
48,2025-07-25-03-00-00 +0000,wsj,As the U.S. heads toward trade talks with Chin...,https://www.wsj.com/economy/trade/trump-china-...,116
226,2025-07-24-17-07-10 +0000,nypost,Trump threatens to escalate trade war with new...,https://nypost.com/2025/07/24/business/trump-t...,111
67,2025-07-25-01-00-00 +0000,wsj,Buoyed by voters’ improving views of the econo...,https://www.wsj.com/politics/policy/trump-appr...,108


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
66,147,2025-07-25-01-00-00 +0000,wsj,Streetwise: The Trump administration has rampe...,https://www.wsj.com/economy/central-banking/tr...
335,87,2025-07-24-07-33-58 +0000,nypost,Thailand and Cambodia exchange fire in new bor...,https://nypost.com/2025/07/24/world-news/thail...
175,85,2025-07-24-19-57-36 +0000,cbc,"Israel, U.S. pull out of Gaza ceasefire talks ...",https://www.cbc.ca/news/world/france-macron-pa...
91,53,2025-07-24-23-50-13 +0000,nypost,Former top Biden White House official blames M...,https://nypost.com/2025/07/24/us-news/former-t...
48,52,2025-07-25-03-00-00 +0000,wsj,As the U.S. heads toward trade talks with Chin...,https://www.wsj.com/economy/trade/trump-china-...
127,47,2025-07-24-21-58-32 +0000,latimes,FCC approves Paramount-Skydance merger followi...,https://www.latimes.com/entertainment-arts/bus...
95,46,2025-07-24-23-28-48 +0000,nypost,Bubba the Love Sponge regrets not making up wi...,https://nypost.com/2025/07/24/us-news/bubba-th...
54,42,2025-07-25-02-10-00 +0000,wsj,Republican National Committee Chair Michael Wh...,https://www.wsj.com/politics/elections/north-c...
268,40,2025-07-24-14-19-43 +0000,nypost,Tesla stock plunges as Elon Musk warns company...,https://nypost.com/2025/07/24/business/tesla-s...
324,40,2025-07-24-09-30-00 +0000,wsj,New York mayoral candidate Zohran Mamdani appe...,https://www.wsj.com/politics/elections/downwar...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
